In [1]:
# Importing the required libraries
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [20]:
test= pd.read_csv(r'C:\Users\harsh\Downloads\test\test.csv')
train = pd.read_csv(r'C:\Users\harsh\Downloads\train\train.csv')

In [21]:
train.head()
# Collect the Y values into an array
y_train = train['y'].values

In [4]:
test.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Understand the data types 
cols = [c for c in train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))
print('Feature types:')
train[cols].dtypes.value_counts()

Number of features: 376
Feature types:


int64     368
object      8
dtype: int64

In [6]:
# Count the data in each of the columns

counts = [[], [], []]
for c in cols:
    typ = train[c].dtype
    uniq = len(np.unique(train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)
print('Constant features: {} Binary features: {} Categorical features: {}\n'
 .format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
print('Categorical features:', counts[2])


Constant features: 5 Binary features: 363 Categorical features: 8

Constant features: ['X257', 'X258', 'X295', 'X296', 'X369']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [22]:
# Splitting the data
usable_columns = list(set(train.columns) - set(['ID', 'y']))
y_train = train['y'].values
id_test = test['ID'].values
x_train = train[usable_columns]
x_test = test[usable_columns]

In [23]:
def check_missing_values(df):
    if df.isnull().any().any():
        print('There are missing values in the dataframe')
    else:
        print('There are no missing values in the dataframe')

In [24]:
check_missing_values(x_train)
check_missing_values(x_test)

There are no missing values in the dataframe
There are no missing values in the dataframe


In [25]:
for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

<ipython-input-25-357911546871>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[column] = x_train[column].apply(mapper)
<ipython-input-25-357911546871>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = x_test[column].apply(mapper)


,X255,X304,X31,X214,X288,X70,X98,X211,X308,X238,...,X79,X108,X373,X341,X205,X291,X204,X335,X371,X228
0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,1,0,0,1,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0,1,1,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0,1,1,0,0,1,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
4,0,1,1,0,0,1,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0


In [31]:
# Make sure the data is changed into numerical values

print('Feature types:')
x_train[cols].dtypes.value_counts()

Feature types:


int64    376
dtype: int64

In [32]:
n_comp = 12
pca = PCA(n_components = n_comp,random_state = 420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

In [33]:
# Training using XGBoost

import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


In [36]:
x_train,x_val,y_train,y_val = train_test_split(pca2_results_train, y_train, test_size=0.2, random_state=4242)

In [37]:
d_train = xgb.DMatrix(x_train,label = y_train)
d_val = xgb.DMatrix(x_val,label = y_val)

# dtest = xgb.DMatrix(x_test)

d_test = xgb.DMatrix(pca2_results_test)

In [38]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)
watchlist = [(d_train, 'train'), (d_val, 'valid')]
clf = xgb.train(params, d_train,  1000, watchlist, early_stopping_rounds=50, 
 feval=xgb_r2_score, maximize=True, verbose_eval=10)

[01:15:13] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.14835	train-r2:-58.35295	valid-rmse:98.26297	valid-r2:-67.63754
[10]	train-rmse:81.27651	train-r2:-38.88428	valid-rmse:80.36433	valid-r2:-44.91014
[20]	train-rmse:66.71610	train-r2:-25.87403	valid-rmse:65.77334	valid-r2:-29.75260
[30]	train-rmse:54.86916	train-r2:-17.17725	valid-rmse:53.89099	valid-r2:-19.64498
[40]	train-rmse:45.24565	train-r2:-11.36019	valid-rmse:44.22210	valid-r2:-12.90147
[50]	train-rmse:37.44744	train-r2:-7.46673	valid-rmse:36.37736	valid-r2:-8.40685
[60]	train-rmse:31.15107	train-r2:-4.85892	valid-rmse:30.01741	valid-r2:-5.40514
[70]	train-rmse:26.08695	train-r2:-3.10883	valid-rmse:24.90702	valid-r2:-3.40987
[80]	train-rmse:22.04893	train-r2:-1.93526	valid-rmse:20.82334	valid-r2:-2.08236
[90]	train-rmse:18.85238	train-r2:-1.14587	valid-rmse:17.59791	valid-r2:-1.20143
[100]	train-rmse:16.34566	

In [39]:
p_test = clf.predict(d_test)

In [40]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('test_df.csv', index = False)
sub.head()

,ID,y
0,1,82.970772
1,2,97.417236
2,3,83.050621
3,4,76.935242
4,5,112.825043
